# L2 - Полносвязанная сеть

### 1. Многоклассовая классификация

Итак, мы знакомы уже с бинарным линейным классификатором
$$y = \text{sign}(wx).$$
Существуют [разные](https://en.wikipedia.org/wiki/Multiclass_classification) подходы к задаче многоклассовой классификции, к примеру [сведение](https://en.wikipedia.org/wiki/Multiclass_classification#Transformation_to_Binary) задачи к бинарной классификации, [модификация модели](https://en.wikipedia.org/wiki/Support_vector_machine#Multiclass_SVM) и т.п. Нам же интересен подход, который применяется в нейронных сетях.

Для каждого класса из набора $1 \dots |C|$ заведем свой вектор весов $w_i$ и уложим это все в матрицу $W$ построчно. Для простоты будем считать, что $w_i$ - строка. Тогда наш классификатор будет выглядеть следующим образом
$$(p_1, \dots, p_{|C|}) = \text{softmax}(Wx),$$
где $p_i$ - вероятность, что объект относится к классу $i$, при этом
$$p_i = \frac{\exp(w_ix)}{\sum_j \exp(w_jx)}.$$
Если внимательно присмотреться, то $\text{softmax}$ является обобщенным вариантом сигмоиды. Для того, чтобы убедиться в этом, достаточно расписать случай для $|C|=2$.

Как и для задачи бинарной классификации, обучение можно свести к минимизации эмпирического риска, то есть к оптимизации следующего функционала
$$\arg\min_W Q(W) = \arg\min_W -\frac{1}{\mathcal{l}}\sum_y\sum_i [y = i] \cdot \ln(p_i(W)).$$
Очевидно, что сверху написано ни что иное, как максимизация логарифма правдоподобия.

**Задание**
1. Вычислите градиент функции $Q$ (попробуйте провести выкладки для отдельной строки $w_i$).
2. Обучите модель с помощью градиентного спуска на выборке [mnist](https://www.kaggle.com/c/digit-recognizer) (вы можете применить свою любимую вариацию метода).
3. Вычислите качество на отоженной выборке.

Давайте кратко каснемся темы регуляризации. Как было рассказано на семинаре, подходы есть разные. Мы же остановимся на модификации оптимизируемого функционала.

$$\arg\min_W -\frac{1}{\mathcal{l}}\sum_y\sum_i [y = i] \cdot \ln(p_i(W)) + \lambda_1 L_1(W) + \lambda_2 L_2(W)$$

1. $L_1(W) = sum_{i,j} |w_{i,j}|$ - пытается занулить бесполезные признаки
2. $L_2(W) = sum_{i,j} w_{i,j}^2$ - не дает параметрам быть слишком большими.

**Задание**
1. Как стоит подбирать значения $\lambda_1$ и $\lambda_2$?
2. Удалось ли улучшить $Q$ на отложенной выборке?

### 2. Полносвязанные нейронные сети

Можно задаться вопросом, а что будет, если поверх модели $Wx$ применить еще одну матрицу, а затем уже $\text{softmax}$? Ответ следующий, дополнительное умножение на матрицу слева $W_2\cdot(W_1x)$ ничего не даст, в итоге это та же самая модель $Wx$, где $W=W_2 W_1$. А вот, если между матрицами добавить третий "слой", то может получиться очень интерсная штука.

Давайте рассмотрим пример на множестве бинарных функций, $x \in \{0, 1\}^{2}$. Пусть есть модель

$$y = \theta(wx + b),$$

где $\theta$ - функция [Хевисайда](https://en.wikipedia.org/wiki/Heaviside_step_function).

**Задание**
1. Предложите значения $w$ и $b$, чтобы $y$ реализовала операторы *and*, *or*, *not*.
2. Приведите пример булевой функции, которая не может быть представлена в виде $y$?

Давайте рассмотрим случай посложнее, пусть теперь $x \in \{0, 1\}^{n}$, а
$$y = \theta(W_2 \cdot \theta(W_1x + b_1) + b_2),$$
где функция $\theta$ для вектора применяется поэлементно. Кстати, судя по размерности, $W_2$ - вектор, но для общности записан с большой буквы.

**Задание**
1. Можете ли вы теперь представить вашу функцию в виде $y$?
2. Может ли $y$ реализовать произвольную булеву функцию? Знаете ли вы, что такое ДНФ?

Теперь мы получили интуицию, почему сочетание нелинейности и нескольких линейных слоев могут дать поразительный эффект! Очевидно, что эту же идею можно переложить и на задачу классификации в пространстве признаков $\mathbb{R}^n$.
Одним словом, мы изобрели полносвязанную нейронную сеть!

Конструкция сети обычно выглядит следующим образом. На вектор признаков $x_1$ сначала применяется линейное преобрзование $W_1$ и некоторая нелинейная функция $\sigma_1$. На выходе мы получаем вектор $x_2$. К нему может быть применена очередная пара $W_2$ и $\sigma_2$ и т.д. Для задачи классификации в частности все заканчивается слоем $\text{softmax}$, которые применяется к выходу $x_{n}$. Теперь, пропустив входной вектор $x_1$ через все эти слои, получим оценку вероятности принадлежности каждому классу.

Некоторые полезные факты:
1. Двуслойная сеть в $\mathbb{R}^n$ позволяет реализовать произвольный выпуклый многогранник.
2. Трехслойная сеть в $\mathbb{R}^n$ позволяет определить уже не обязательно выпуклую и даже не обязательно связанную область.
3. С помощью линейных операций и фиксированной функции $\sigma$ можно приблизить функцию с коль угодной точностью.

### 3. Обучение

Осталось только понять, как все эти слои обучить. Для этого нам понадобиться ввести кое-какие обозначения. Пусть есть некоторая функция, которая на вход принимает вектор $x = (x_1, \dots, x_n)$ и выдает $y = (y_1, \dots, y_m)$, то есть размерности могу не совпадать, тогда

$$\frac{dy}{dx} = \Big[g_{ij} = \frac{\partial y_j}{\partial x_i}\Big].$$

Не нарушая общности можно предположить, что каждый слой это функция $$f_i(x_i, w_i) = x_{i+1}.$$ При этом для обучения в самый конец добавляется слой с вычислением ошибки. Таким образом всю работу такой сети можно обозначить за $Q$, которая принимает $x_1$ и возвращает некоторое значение ошибки.

**Forward step**

Первым делом просто запускаем вычисления по слоям и узнаем все значения $x_i$. Тут все просто и справится даже школьник.

**Backward step**

Рассмотрим последний слой, градиентный шаг для $w_n$ вычисляется очевидным образом $\frac{dQ}{dw_n}$. При этом на всякий случай вычислим еще и $\frac{dQ}{dx_{n}}$. 

Теперь на необходимо посчитать то же самое на слой ниже, но на семинаре мы подробно обсуждали, что 
$$\frac{dQ}{dw_{n-1}} = \frac{dx_{n}}{dw_{n-1}}\frac{dQ}{dx_{n}} \text{ и } \frac{dQ}{dx_{n-1}} = \frac{dx_{n}}{dx_{n-1}}\frac{dQ}{dx_{n}}.$$
То есть теперь мы вычислили градиент для парамтеров слоем ниже. Повторяя таки образом операции, вычислим $\frac{dQ}{dw_{n-2}}$ и $\frac{dQ}{dx_{n-2}}$ и т.д.

Теперь, когда градиент найден, мы можем спокойно применить метод градиентного спуска для обучения весов. Стоит отметить, что сети обучают именно таким подходом, при этом для лучшего обучения повсеместно применяется батчинг.

**Моменты реализации**

Итак, нейросеть удобно реализовывать в виде слоев $y=f(x, w)$, где у каждого слоя есть следующие методы:
* $f$ - вычисляет значение слоя для  текущего значения $w$ и вектора $x$.
* $dydw$ - метод необходимый для вычисление шага градиентного спуска
* $dydx$ - метод необходим для вычисления шага для слоя ниже
* $update$ -  обновляет параметр $w$ для слоя на указанную величину

Тогда, если есть список слоев и определена функция потерь последним дополнительным слоем, то обучение сводиться к запуска forward вычислений, а затем постепенном обратном распространении ошибки и обновления параметров, то есть шаг $backward$. Будьте аккуратны при реализации.

**Задание**

1. Ознакомьтесь с библиотекой [Keras](https://keras.io). Примеры, которые были разобраны на семинаре
 * [Полносвязанная сеть](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py)
 * [Сверточная сеть](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py)
 * [Переобучение сверточной сети](https://github.com/fchollet/keras/blob/master/examples/mnist_transfer_cnn.py)
2. Реализуйте двухслойную сеть, где в качестве нелинейной функции используется [ReLu](https://en.wikipedia.org/wiki/Rectifier_(neural_networks).
3. Улучшилось ли качество в сравнении с предыдущей моделью?
4. Какова размерность выходного вектора после первого линейного преобразования? Как влияет его размер? Постройте график.
5. Предложите свою архитектуру
6. Как зависит качество от количества слоев? Постройте график.
7. Попробуйте постепенно добавлять слои в вашу сеть, для это изучите следующие трюки:
 * [Autoencoder](https://en.wikipedia.org/wiki/Autoencoder)
 * [RBM](https://en.wikipedia.org/wiki/Restricted_Boltzmann_machine)